# Szybka wyszukiwarka korzystająca z SVD

In [1]:
from datasets import load_dataset
import pickle
from nltk.stem import PorterStemmer
import string
import enchant
from scipy import sparse
from math import log
from sklearn.preprocessing import normalize
from sklearn.utils.extmath import randomized_svd
import scipy

/home/jakub/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pobieram stopwords w celu odfiltrowania ich z przeszukiwanych tekstów

In [16]:
import nltk
nltk.download('stopwords')
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /home/jakub/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jakub/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from nltk.corpus import stopwords
english_dict = enchant.Dict("en_US")
stopwords = stopwords.words("english")

## Korzystam z gotowego datasetu (dumpu wikipedii z 2022 roku)

In [19]:
dataset = load_dataset("wikipedia", "20220301.en")

Generating train split: 100%|██████████| 6458670/6458670 [01:05<00:00, 98795.34 examples/s] 


In [21]:
dataset = dataset["train"]

In [24]:
new_dataset = []
key_words = ["war", "doom", "death", "army", "artillery", "weapon", "soldier", "battle", "bomb", "gun", "military", "combat", "fight", "kill", "injury", "casualty", "violence", "attack", "defense", "peace", "nuclear", "missile", "aircraft", "navy", "marine", "infantry", "cavalry", "artillery", "tank", "airforce", "submarine", "grenade", "rifle", "bullet", "shell", "explosion", "fire", "smoke", "blood", "wound", "suffer", "pain", "fear", "terror", "horror", "atrocity", "massacre"]

for site in dataset:
    tmp_text = site["text"].split(" ")
    for word in key_words:
        if word in tmp_text:
            new_dataset.append(site)
            break
    if len(new_dataset) > 100_000:
        break

In [25]:
dataset = new_dataset

## Zapisuje przygotowany dataset do pliku

In [27]:
with open('dataset/my_dataset.bin', 'wb') as file:
    pickle.dump(dataset, file)

In [3]:
with open('dataset/my_dataset.bin', 'rb') as file:
    dataset = pickle.load(file)

## Indeksuje przygotowany dataset

In [4]:
for i in range(len(dataset)):
    dataset[i]["id"] = i

## Upraszczam teksty artykułów korzystając z PorterStemmera

In [5]:
def simplify_string(text: str) -> list:
    def my_filter(x):
        if len(x) < 2 or any(char.isdigit() for char in x) or not english_dict.check(x):
            return False
        return True

    ps = PorterStemmer()
    text = text.lower().translate(str.maketrans("\n", " ", string.punctuation)).split(" ")
    text = [word for word in text if word not in stopwords]
    text = list(filter(my_filter, text))
    text = list(map(ps.stem, text))
    return text

### Określam słownik słów kluczowych (termów) potrzebny do wyznaczenia wektorów  
### cech bag-of-words (indeksacja).  
### Dla każdego dokumentu j wyznaczam wektor cech bag-of-words dj zawierający czę-  
### stości występowania poszczególnych słów (termów) w tekście.   

In [31]:
all_words = set()
words_per_article = list()
word_occurance = dict()

In [32]:
for i in range(len(dataset)):
    text = simplify_string(dataset[i]["text"])
    word_counter = dict()
    for word in text:
        if word not in word_counter:
            word_counter[word] = 1
        else:
            word_counter[word] += 1
    for word in word_counter:
        all_words.add(word)
        if word not in word_occurance:
            word_occurance[word] = 1
        else:
            word_occurance[word] += 1
    words_per_article.append(word_counter)

SAVING for i = 0
0 / 100001
1 / 100001
2 / 100001
3 / 100001
4 / 100001
5 / 100001
6 / 100001
7 / 100001
8 / 100001
9 / 100001
10 / 100001
11 / 100001
12 / 100001
13 / 100001
14 / 100001
15 / 100001
16 / 100001
17 / 100001
18 / 100001
19 / 100001
20 / 100001
21 / 100001
22 / 100001
23 / 100001
24 / 100001
25 / 100001
26 / 100001
27 / 100001
28 / 100001
29 / 100001
30 / 100001
31 / 100001
32 / 100001
33 / 100001
34 / 100001
35 / 100001
36 / 100001
37 / 100001
38 / 100001
39 / 100001
40 / 100001
41 / 100001
42 / 100001
43 / 100001
44 / 100001
45 / 100001
46 / 100001
47 / 100001
48 / 100001
49 / 100001
50 / 100001
51 / 100001
52 / 100001
53 / 100001
54 / 100001
55 / 100001
56 / 100001
57 / 100001
58 / 100001
59 / 100001
60 / 100001
61 / 100001
62 / 100001
63 / 100001
64 / 100001
65 / 100001
66 / 100001
67 / 100001
68 / 100001
69 / 100001
70 / 100001
71 / 100001
72 / 100001
73 / 100001
74 / 100001
75 / 100001
76 / 100001
77 / 100001
78 / 100001
79 / 100001
80 / 100001
81 / 100001
82 / 1000

### Zapisuje te dane do plików

In [34]:
with open('dataset/all_words.bin', 'wb') as file:
    pickle.dump(all_words, file)
with open('dataset/words_per_article.bin', 'wb') as file:
    pickle.dump(words_per_article, file)
with open('dataset/word_occurance.bin', 'wb') as file:
    pickle.dump(word_occurance, file)

In [6]:
with open('dataset/all_words.bin', 'rb') as file:
    all_words = pickle.load(file)
with open('dataset/words_per_article.bin', 'rb') as file:
    words_per_article = pickle.load(file)
with open('dataset/word_occurance.bin', 'rb') as file:
    word_occurance = pickle.load(file) 

In [7]:
all_words = sorted(list(all_words))
all_words_indexes = dict()
for i, word in enumerate(all_words):
    all_words_indexes[word] = i

In [8]:
N = len(dataset)
M = len(all_words)

### Buduję rzadką macierz wektorów cech term-by-document matrix w której wekto-  
### ry cech ułożone są kolumnowo Am×n = [d1|d2|...|dn] (m jest liczbą termów w  
### słowniku, a n liczbą dokumentów)  

### Przetwarzam wstępnie otrzymany zbiór danych mnożąc elementy bag-of-words przez  
### inverse document frequency. Operacja ta pozwoli na redukcję znaczenia często wy-  
### stępujących słów.  
### IDF(w) = log N/nw,
### gdzie nw jest liczbą dokumentów, w których występuje słowo w, a N jest całkowitą
### liczbą dokumentów.

In [22]:
def save_sparse_mat_file(sparse_matrix, filename):
    sparse.save_npz(f"dataset/{filename}", sparse_matrix)

def create_sparse_matrix_for_all_articles(use_idf = False):
    row = []
    col = []
    word_counter = []
    for i in range(N):
        dict_of_words = words_per_article[i]
        for word in dict_of_words:
            col.append(i)
            row.append(all_words_indexes[word])
            if use_idf:
                word_counter.append(dict_of_words[word] * log(N / word_occurance[word]))
            else:
                word_counter.append(dict_of_words[word])
    return sparse.csr_matrix((word_counter, (row, col)), shape=(M, N))


def create_single_vector(text: str):
    text = simplify_string(text)
    tmp = []
    for word in text:
        if word in all_words:
            tmp.append(word)
    text = tmp

    word_counter = dict()
    for word in text:
        if word not in word_counter:
            word_counter[word] = 1
        else:
            word_counter[word] += 1

    row = []
    col = []

    data = []
    for word in word_counter:
        row.append(0)
        col.append(all_words_indexes[word])
        data.append(word_counter[word])
    return sparse.csr_matrix((data, (row, col)), shape=(1, M))


def return_k_nearest_articles(searched_phrase, matrix_of_words, k = 10):
    q = create_single_vector(searched_phrase)
    q_norm = sparse.linalg.norm(q, axis=1)
    ans = []
    min_val = 0
    for i in range(N):
        col = matrix_of_words.getcol(i)
        val = q * col
        if not val:
            continue
        val = (val.data / (q_norm * sparse.linalg.norm(col, axis=0)))[0]
        if len(ans) < k:
            ans.append((i, val))
        elif val > min_val:
            ans.append((i, val))
            ans.sort(key = lambda x: x[1], reverse=True)
            ans.pop(-1)
            min_val = ans[-1][1]
    ans = [index[0] for index in ans]
    return ans

def return_k_nearest_articles_normalized(searched_phrase, k = 10):
    q = create_single_vector(searched_phrase)
    q_normalized = normalize(q, norm="l1", axis=1)
    ans_mat = q_normalized @ sparse_mat_normalized
    cx = ans_mat.tocoo()
    ans = []
    min_val = 0
    for i, val in zip(cx.col, cx.data):
        if len(ans) < k:
            ans.append((i, val))
        elif val > min_val:
            ans.append((i, val))
            ans.sort(key = lambda x: x[1], reverse=True)
            ans.pop(-1)
            min_val = ans[-1][1]
    ans = [index[0] for index in ans]
    return ans

def return_k_nearest_articles_svd(searched_phrase, svd_size, k = 10):
    q = create_single_vector(searched_phrase)
    q_normalized = normalize(q, norm="l1", axis=1)

    with open(f'dataset/sparse_svd_U_{svd_size}.bin', 'rb') as file:
        svd_U = pickle.load(file)
        temp = q_normalized @ svd_U
        del svd_U

    with open(f'dataset/sparse_svd_D_{svd_size}.bin', 'rb') as file:
        svd_D = pickle.load(file)
        temp = temp @ sparse.diags(svd_D)
        del svd_D

    with open(f'dataset/sparse_svd_V_{svd_size}.bin', 'rb') as file:
        svd_V = pickle.load(file)
        ans_mat = temp @ svd_V
        del svd_V, temp

    ans_mat = sparse.csr_matrix(ans_mat)
    cx = ans_mat.tocoo()
    ans = []
    min_val = 0
    for i, val in zip(cx.col, cx.data):
        if len(ans) < k:
            ans.append((i, val))
        elif val > min_val:
            ans.append((i, val))
            ans.sort(key = lambda x: x[1], reverse=True)
            ans.pop(-1)
            min_val = ans[-1][1]
    ans = [index[0] for index in ans]
    return ans

### Budowa macierzy rzadkich i SVD

In [39]:
sparse_mat = create_sparse_matrix_for_all_articles(False)
save_sparse_mat_file(sparse_mat, "sparse_mat_default.npz")

In [40]:
sparse_mat_idf = create_sparse_matrix_for_all_articles(True)
save_sparse_mat_file(sparse_mat_idf, "sparse_mat_idf.npz")

In [41]:
sparse_mat_normalized = normalize(sparse_mat_idf, norm="l1", axis=0)
save_sparse_mat_file(sparse_mat_normalized, "sparse_mat_normalized.npz")

In [42]:
mat_U, mat_D, mat_V = randomized_svd(sparse_mat_normalized, n_components=1000, random_state=None)
with open(f'dataset/sparse_svd_U_1000.bin', 'wb') as file:
    pickle.dump(mat_U, file)
with open(f'dataset/sparse_svd_D_1000.bin', 'wb') as file:
    pickle.dump(mat_D, file)
with open(f'dataset/sparse_svd_V_1000.bin', 'wb') as file:
    pickle.dump(mat_V, file)

In [10]:
sparse_mat = sparse.load_npz("dataset/sparse_mat_default.npz")
sparse_mat_idf = sparse.load_npz("dataset/sparse_mat_idf.npz")
sparse_mat_normalized = sparse.load_npz("dataset/sparse_mat_normalized.npz")

## Testy

In [11]:
k_nearest_svd = return_k_nearest_articles_svd("many people died then", 1000)
for i, index in enumerate(k_nearest_svd):
    print(f"{i+1}: {dataset[index]['title']} | {dataset[index]['url']}")

1: Margaret of Huntingdon, Lady of Galloway | https://en.wikipedia.org/wiki/Margaret%20of%20Huntingdon%2C%20Lady%20of%20Galloway
2: Die | https://en.wikipedia.org/wiki/Die
3: 1797 in science | https://en.wikipedia.org/wiki/1797%20in%20science
4: Razi | https://en.wikipedia.org/wiki/Razi
5: 1628 in science | https://en.wikipedia.org/wiki/1628%20in%20science
6: Pehr Adlerfelt | https://en.wikipedia.org/wiki/Pehr%20Adlerfelt
7: 1891 in film | https://en.wikipedia.org/wiki/1891%20in%20film
8: 1708 in science | https://en.wikipedia.org/wiki/1708%20in%20science
9: 1500 in literature | https://en.wikipedia.org/wiki/1500%20in%20literature
10: 1826 in literature | https://en.wikipedia.org/wiki/1826%20in%20literature


In [24]:
k_nearest_svd = return_k_nearest_articles_svd("war crimes", 1000)
for i, index in enumerate(k_nearest_svd):
    print(f"{i+1}: {dataset[index]['title']} | {dataset[index]['url']}")

1: Egyptian War | https://en.wikipedia.org/wiki/Egyptian%20War
2: List of wars between Russia and Sweden | https://en.wikipedia.org/wiki/List%20of%20wars%20between%20Russia%20and%20Sweden
3: War of independence | https://en.wikipedia.org/wiki/War%20of%20independence
4: Russo-German war | https://en.wikipedia.org/wiki/Russo-German%20war
5: Northern Wars | https://en.wikipedia.org/wiki/Northern%20Wars
6: Battle of Dunbar | https://en.wikipedia.org/wiki/Battle%20of%20Dunbar
7: List of battles (alphabetical) | https://en.wikipedia.org/wiki/List%20of%20battles%20%28alphabetical%29
8: List of civil wars | https://en.wikipedia.org/wiki/List%20of%20civil%20wars
9: List of military divisions | https://en.wikipedia.org/wiki/List%20of%20military%20divisions
10: Pre-war | https://en.wikipedia.org/wiki/Pre-war


In [13]:
k_nearest_svd = return_k_nearest_articles_svd("nazist crimes", 1000)
for i, index in enumerate(k_nearest_svd):
    print(f"{i+1}: {dataset[index]['title']} | {dataset[index]['url']}")

1: Violent crime | https://en.wikipedia.org/wiki/Violent%20crime
2: Gang war | https://en.wikipedia.org/wiki/Gang%20war
3: Perfect crime | https://en.wikipedia.org/wiki/Perfect%20crime
4: Crime statistics | https://en.wikipedia.org/wiki/Crime%20statistics
5: Crimes against humanity | https://en.wikipedia.org/wiki/Crimes%20against%20humanity
6: National Crime Victimization Survey | https://en.wikipedia.org/wiki/National%20Crime%20Victimization%20Survey
7: Drug cartel | https://en.wikipedia.org/wiki/Drug%20cartel
8: True crime | https://en.wikipedia.org/wiki/True%20crime
9: Justification (jurisprudence) | https://en.wikipedia.org/wiki/Justification%20%28jurisprudence%29
10: War Crimes Act 1991 | https://en.wikipedia.org/wiki/War%20Crimes%20Act%201991


In [14]:
k_nearest_svd = return_k_nearest_articles_svd("is a crime in which an offender or perpetrator uses or threatens to use harmful force upon a victim", 1000)
for i, index in enumerate(k_nearest_svd):
    print(f"{i+1}: {dataset[index]['title']} | {dataset[index]['url']}")

1: RCAF (disambiguation) | https://en.wikipedia.org/wiki/RCAF%20%28disambiguation%29
2: Australian Imperial Force | https://en.wikipedia.org/wiki/Australian%20Imperial%20Force
3: Violent crime | https://en.wikipedia.org/wiki/Violent%20crime
4: Crime statistics | https://en.wikipedia.org/wiki/Crime%20statistics
5: List of aircraft of the South African Air Force | https://en.wikipedia.org/wiki/List%20of%20aircraft%20of%20the%20South%20African%20Air%20Force
6: Perfect crime | https://en.wikipedia.org/wiki/Perfect%20crime
7: Kerem Yılmazer | https://en.wikipedia.org/wiki/Kerem%20Y%C4%B1lmazer
8: Crimes against humanity | https://en.wikipedia.org/wiki/Crimes%20against%20humanity
9: National Crime Victimization Survey | https://en.wikipedia.org/wiki/National%20Crime%20Victimization%20Survey
10: Justification (jurisprudence) | https://en.wikipedia.org/wiki/Justification%20%28jurisprudence%29


In [11]:
k_nearest_normalized = return_k_nearest_articles_normalized("war crimes", 10)
for i, index in enumerate(k_nearest_normalized):
    print(f"{i+1}: {dataset[index]['title']} | {dataset[index]['url']}")

1: Egyptian War | https://en.wikipedia.org/wiki/Egyptian%20War
2: List of wars between Russia and Sweden | https://en.wikipedia.org/wiki/List%20of%20wars%20between%20Russia%20and%20Sweden
3: War of independence | https://en.wikipedia.org/wiki/War%20of%20independence
4: Russo-German war | https://en.wikipedia.org/wiki/Russo-German%20war
5: Northern Wars | https://en.wikipedia.org/wiki/Northern%20Wars
6: List of battles (alphabetical) | https://en.wikipedia.org/wiki/List%20of%20battles%20%28alphabetical%29
7: Battle of Dunbar | https://en.wikipedia.org/wiki/Battle%20of%20Dunbar
8: List of civil wars | https://en.wikipedia.org/wiki/List%20of%20civil%20wars
9: List of military divisions | https://en.wikipedia.org/wiki/List%20of%20military%20divisions
10: List of wars involving Finland | https://en.wikipedia.org/wiki/List%20of%20wars%20involving%20Finland
